In [1]:
import pandas as pd
import numpy as np
import regex as re
import json

In [2]:
import sklearn
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.cluster import KMeans
from sklearn.preprocessing import normalize

Загружаем с раздела "открытые данные" ресурса trudvsem.ru датасет по всем вакансия в России. С данного датасета необходимо отобрать вакансии, относящиеся к Data Science

In [3]:
df_ = pd.read_csv("./raw_dataset/vacancy.csv", sep="|")
df_.head()

C:\Users\Andrew\AppData\Local\Temp\ipykernel_7808\3929292859.py:1: DtypeWarning: Columns (1,69) have mixed types. Specify dtype option on import or set low_memory=False.
  df_ = pd.read_csv("./raw_dataset/vacancy.csv", sep="|")


,id,academic_degree,accommodation_capability,accommodation_type,additional_premium,additional_requirements,bonus_type,measure_type,busy_type,career_perspective,...,federalDistrictCode,industryBranchName,contactList,company_name,full_company_name,company_inn,company,languageKnowledge,hardSkills,softSkills
0,89bbb378-2c6a-11ef-b595-cb26dff57dd7,NaN,False,NaN,NaN,NaN,NaN,NaN,Полная занятость,False,...,4.0,NaN,NaN,NaN,"МУНИЦИПАЛЬНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ УЧРЕЖДЕНИЕ ""...",7.313003e+09,"{""companycode"":""1027300870288"",""email"":""kopt85...",[],[],[]
1,02a867b8-47d2-11ef-8517-cb26dff57dd7,NaN,False,NaN,NaN,NaN,NaN,NaN,Частичная занятость,False,...,6.0,NaN,NaN,NaN,МУНИЦИПАЛЬНОЕ БЮДЖЕТНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ УЧ...,5.404164e+09,"{""companycode"":""1025401494557"",""email"":""s_14@e...",[],[],[]
2,029c8645-cc5d-11ec-90ec-550ed7335bbe,NaN,False,NaN,NaN,NaN,NaN,NaN,Полная занятость,False,...,1.0,NaN,NaN,NaN,"АКЦИОНЕРНОЕ ОБЩЕСТВО ""ТАМБОВМАШ""",6.829000e+09,"{""companycode"":""1026801223679"",""email"":""tambov...",[],[],[]
3,027a4355-a83a-11ed-b894-05dc90903fb8,NaN,False,NaN,40.0,NaN,MONTHLY,PERCENT,Полная занятость,False,...,7.0,NaN,NaN,NaN,Дальэнергосбыт,2.723089e+09,"{""companycode"":""97d8b820-f029-11eb-8576-3bfa22...",[],[],[]
4,02759145-7402-11ee-b876-a3cd0470df2e,NaN,True,DORMITORY,25.0,NaN,MONTHLY,PERCENT,Полная занятость,False,...,4.0,NaN,NaN,NaN,"ФЕДЕРАЛЬНОЕ КАЗЕННОЕ ПРЕДПРИЯТИЕ ""ПЕРМСКИЙ ПОР...",5.908006e+09,"{""companycode"":""1025901604156"",""email"":""ppz@pe...",[],[],[]


In [4]:
df= df_.copy()

Проводим разведочный анализ исходного датасета с ресурса trudvsem.ru

In [5]:
df.shape

(627839, 78)

In [6]:
df.drop_duplicates(inplace=True)
df.shape

(627839, 78)

In [7]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627839 entries, 0 to 627838
Data columns (total 78 columns):
 #   Column                           Non-Null Count   Dtype  
---  ------                           --------------   -----  
 0   id                               627839 non-null  object 
 1   academic_degree                  343 non-null     object 
 2   accommodation_capability         627839 non-null  bool   
 3   accommodation_type               30642 non-null   object 
 4   additional_premium               74699 non-null   float64
 5   additional_requirements          221035 non-null  object 
 6   bonus_type                       75254 non-null   object 
 7   measure_type                     75006 non-null   object 
 8   busy_type                        627839 non-null  object 
 9   career_perspective               627839 non-null  bool   
 10  change_time                      623666 non-null  object 
 11  code_external_system             108860 non-null  object 
 12  co

СОздам словарь соотношения кода федерального к региону

In [43]:
cod_reg_fed = df[["regionName","federalDistrictCode"]].apply(lambda x: ":".join(x.values.astype(str)), axis=1)

In [44]:
cod_reg_fed.unique()

array(['Ульяновская область:4.0', 'Новосибирская область:6.0',
       'Тамбовская область:1.0', 'Приморский край:7.0',
       'Пермский край:4.0', 'Республика Башкортостан:4.0',
       'Московская область:1.0', 'Республика Бурятия:7.0',
       'Челябинская область:5.0', 'Свердловская область:5.0',
       'Сахалинская область:7.0', 'Саратовская область:4.0',
       'Удмуртская Республика:4.0', 'Смоленская область:1.0',
       'Иркутская область:6.0', 'Город Москва:1.0',
       'Красноярский край:6.0', 'Пензенская область:4.0',
       'Краснодарский край:3.0', 'Город Санкт-Петербург:2.0',
       'Ярославская область:1.0', 'Республика Карелия:2.0',
       'Кемеровская область - Кузбасс:6.0', 'Самарская область:4.0',
       'Курганская область:5.0', 'Астраханская область:3.0',
       'Нижегородская область:4.0', 'Ставропольский край:8.0',
       'Амурская область:7.0', 'Республика Дагестан:8.0',
       'Ростовская область:3.0', 'Тверская область:1.0',
       'Чувашская Республика - Чувашия

In [8]:
df.dtypes.unique()

array([dtype('O'), dtype('bool'), dtype('float64'), dtype('int64')],
      dtype=object)

Выявляем колонки с указанием заработной платы ('salary', 'salary_min', 'salary_max') и отфильтовываем датасет от отсутсвующих значений з/п, т.е вакансии указанные без заработной платы или с нулевым значением для нас не представляют интерес (возможно, можно будет вернуться в дальнейшем к ним, так как предполагается, что сейчас много вакансий отфильтруется из-за того, что в сфере ИТ очень сильно распространена практика размещать вакансии без указания уровня з/п).

In [9]:
df.columns

Index(['id', 'academic_degree', 'accommodation_capability',
       'accommodation_type', 'additional_premium', 'additional_requirements',
       'bonus_type', 'measure_type', 'busy_type', 'career_perspective',
       'change_time', 'code_external_system', 'code_profession',
       'code_professional_sphere', 'company_code', 'contact_person',
       'contact_source', 'data_ids', 'date_create', 'date_modify', 'deleted',
       'education', 'education_speciality', 'foreign_workers_capability',
       'metro_ids', 'is_mobility_program', 'is_moderated',
       'is_uzbekistan_recruitment', 'is_quoted', 'need_medcard', 'oknpo_code',
       'okso_code', 'original_source_type', 'other_vacancy_benefit',
       'position_requirements', 'position_responsibilities',
       'publication_period', 'published_date', 'regionName', 'regionNameTerm',
       'company_business_size', 'required_certificates',
       'required_drive_license', 'required_experience',
       'retraining_capability', 'retraining_

Выявляем колонки название которых содержит значения уровня з/п

In [10]:
[col for col in df.columns if "salary" in col]

['salary', 'salary_min', 'salary_max']

In [11]:
df[['salary', 'salary_min', 'salary_max']].info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 627839 entries, 0 to 627838
Data columns (total 3 columns):
 #   Column      Non-Null Count   Dtype  
---  ------      --------------   -----  
 0   salary      616370 non-null  object 
 1   salary_min  616370 non-null  float64
 2   salary_max  612989 non-null  float64
dtypes: float64(2), object(1)
memory usage: 14.4+ MB


Выбираем строки, в которых значения з/п отсутсвует

In [12]:
index_zezo_salazy = df[(df.salary_min == 0)&(df.salary_max == 0)].index

In [13]:
df = df[df.index.map(lambda x: x not in index_zezo_salazy)]
df.shape

(615242, 78)

In [14]:
df = df[~df['salary'].isnull() | ~df['salary_min'].isnull() | ~df['salary_max'].isnull()]
df.shape

(604047, 78)

Исходный датасет сократился на  23 792 (627839 - 604047) вакансий после фильтрации по наличию уровня заработной платы

Проведем анализ размещения вакансий по дате

In [15]:

[col for col in df.columns if "date" in col]

['date_create', 'date_modify', 'published_date']

In [16]:
[col for col in df.columns if "time" in col]

['change_time']

In [17]:
df[['change_time','date_create','date_modify',"published_date"]] =  df[['change_time','date_create','date_modify',"published_date"]].apply(
    lambda x: pd.to_datetime(x,utc = True))

Анализируем распределение всех вакансий по годам публикации

In [18]:
df["published_date"].dt.year.value_counts()

published_date
2024    603969
2023        56
2022        21
2016         1
Name: count, dtype: int64

Оставим вакансии опубликованные в 2024 году

In [19]:
df = df[df["published_date"].dt.year == 2024]
df.shape

(603969, 78)

Далее необходимо отобрать вакансии, относящиеся к Data Science. И провести анализ данных с целью сокращения количества столбцов, так как сейчас их 78. По названию колонок, предварительно, можно понять, что они не все нужны для итогового анализа, но удаление надо производить, когда отберу вакансии, содержащие должности/специализации храктерные для области DS.

In [20]:
col_obj = df.select_dtypes(include=['object']).columns
col_obj.__len__(), col_obj

(47,
 Index(['id', 'academic_degree', 'accommodation_type',
        'additional_requirements', 'bonus_type', 'measure_type', 'busy_type',
        'code_external_system', 'code_professional_sphere', 'company_code',
        'contact_person', 'contact_source', 'data_ids', 'education',
        'education_speciality', 'metro_ids', 'is_mobility_program',
        'need_medcard', 'original_source_type', 'other_vacancy_benefit',
        'position_requirements', 'position_responsibilities', 'regionName',
        'company_business_size', 'required_certificates',
        'required_drive_license', 'retraining_capability', 'retraining_grant',
        'salary', 'schedule_type', 'social_protected_ids', 'source_type',
        'status', 'transport_compensation', 'vacancy_address_additional_info',
        'vacancy_address', 'vacancy_address_house', 'vacancy_benefit_ids',
        'vacancy_name', 'visibility', 'professionalSphereName',
        'industryBranchName', 'full_company_name', 'company',
        '

In [21]:
df[col_obj].describe().transpose()

,count,unique,top,freq
id,603969,603969,89bbb378-2c6a-11ef-b595-cb26dff57dd7,1
academic_degree,338,2,Кандидат наук,281
accommodation_type,30529,4,DORMITORY,18703
additional_requirements,219429,141277,Ответственность,6780
bonus_type,75004,3,MONTHLY,52996
measure_type,74755,2,PERCENT,57224
busy_type,603969,6,Полная занятость,552780
code_external_system,88535,3,CZN,56537
code_professional_sphere,603965,36,Education,84872
company_code,603969,105575,5147746474134,19196


Из визуального анализа названия колонок, пытыюсь найти колонки, содержащие классификатор вакансий. При дальнейшем анализе содержимых колонок, пришел к выводу, что так как тут представлены вакансии по всем профессиям и специализациям в России, необходимо по ключевым выражениям отбирать вакансии, относящиеся по специфики решения задач к Data Science, так как даже в индустрии лесоохраны присутсвуют вакансии для соискателей с навыками работы из области CV.

In [22]:
df.professionalSphereName.value_counts()

professionalSphereName
Образование, наука                                                  84872
Здравоохранение и социальное обеспечение                            82502
Производство                                                        75130
Продажи, закупки, снабжение, торговля                               34852
ЖКХ, эксплуатация                                                   34345
Транспорт, автобизнес, логистика, склад, ВЭД                        28491
Строительство, ремонт, стройматериалы, недвижимость                 21694
Сельское хозяйство, экология, ветеринария                           19541
Работы, не требующие квалификации                                   19252
Пищевая промышленность                                              18546
Рабочие специальности                                               18435
Электроэнергетика                                                   15208
Машиностроение                                                      14253
Государственная

In [23]:
df.industryBranchName.value_counts()

industryBranchName
Авиационная промышленность                     305
Радиопромышленность                            274
Промышленность обычных вооружений              272
Электронная промышленность                     249
Промышленность боеприпасов и спецхимии         235
Судостроительная промышленность                144
Промышленность средств связи                   101
Атомная промышленность                          43
Гражданские отрасли промышленности ВТК          42
Ракетно-космическая промышленность              26
Межотраслевые предприятия и организации ВТК      6
Name: count, dtype: int64

In [24]:
df.vacancy_name.value_counts()


vacancy_name
Продавец-кассир                                                                  7374
Повар                                                                            7269
Бухгалтер                                                                        5540
Водитель автомобиля                                                              5309
Подсобный рабочий                                                                4759
                                                                                 ... 
Специалист по ГО ЧС, пожарной безопасности, антитеррористической защищенности       1
Машинист агрегатов по обслуживанию нефтепромыслового оборудования (АРОК)            1
Инженер-конструктор (разработка и сопровождение БМ с пушечным вооружением)          1
заместитель заведующего по учебно-производственной работе                           1
Электромонтер по лифтам                                                             1
Name: count, Length: 136437, dtype: int64

Далее в колонке содержащей код професии выявлен класс, относящийся к ИТ с почти 8 800 вакансиями, которые содержат значения з/п. Но в дальнейшем при отборе необходимых вакансий, относящихся к Data Science, по ключевым выражениям значительно сократится количество вакансий.

In [25]:
df.code_professional_sphere.value_counts()

code_professional_sphere
Education                    84872
Medicine                     82502
Industry                     75130
Sales                        34852
Communal                     34345
Transport                    28491
BuldindRealty                21694
Agricultural                 19541
NotQualification             19252
Food                         18546
WorkingSpecialties           18435
ElectricpowerIndustry        15208
MechanicalEngineering        14253
StateServices                13601
Safety                       12541
DeskWork                     11991
Restaurants                  11752
Culture                      10775
Resources                     9414
AccountingTaxesManagement     9174
InformationTechnology         8779
ServiceMaintenance            8002
Metallurgy                    7851
Forest                        7623
ChemicalAndFuelIndustry       4478
RootLightIndustry             4389
Finances                      4114
Jurisprudence                 

Необходимо по регулярным выражениям осуществить отбор вакансий, которые содержат название специализаций из области DS. На ресурсах GitHub и Kaggle нашел небольшие датасеты, содержащие название должностей/специализаций, относящиеся к DS. Эти данные с небольшим дополнением специфичных требований (т.е. навыков) для DS были использованы для отбора необходимых вакансий из нашего иcходного датасета.

Отбираем с датасета (ds_vacancies.csv), содержащего вакансии DS с США, Канады и др.стран за период 21-23 г.г. наименование специализаций

In [26]:
df_ds_vac = pd.read_csv("./raw_dataset/Kaggle_dataset/ds_salaries.csv", sep=",")
df_ds_vac.head()

,work_year,experience_level,employment_type,job_title,salary,salary_currency,salary_in_usd,employee_residence,remote_ratio,company_location,company_size
0,2023,SE,FT,Principal Data Scientist,80000,EUR,85847,ES,100,ES,L
1,2023,MI,CT,ML Engineer,30000,USD,30000,US,100,US,S
2,2023,MI,CT,ML Engineer,25500,USD,25500,US,100,US,S
3,2023,SE,FT,Data Scientist,175000,USD,175000,CA,100,CA,M
4,2023,SE,FT,Data Scientist,120000,USD,120000,CA,100,CA,M


In [34]:
df_ds_vac.experience_level.unique()

array(['SE', 'MI', 'EN', 'EX'], dtype=object)

In [36]:
df_ds_vac["job_title"][df_ds_vac.experience_level=='MI']

1                              ML Engineer
2                              ML Engineer
19                            Data Analyst
20                            Data Analyst
35                           Data Engineer
                       ...                
3743                         Data Engineer
3746                        Data Scientist
3747    Applied Machine Learning Scientist
3748                         Data Engineer
3751              Principal Data Scientist
Name: job_title, Length: 805, dtype: object

In [27]:
df_ds_vac.job_title.str.lower().nunique()

93

Получен список из 93 наименований специальностей из области DS на английском языке. 

In [142]:
list_job_ds_eng = df_ds_vac.job_title.str.lower().unique()
list_job_ds_eng

array(['principal data scientist', 'ml engineer', 'data scientist',
       'applied scientist', 'data analyst', 'data modeler',
       'research engineer', 'analytics engineer',
       'business intelligence engineer', 'machine learning engineer',
       'data strategist', 'data engineer', 'computer vision engineer',
       'data quality analyst', 'compliance data analyst',
       'data architect', 'applied machine learning engineer',
       'ai developer', 'research scientist', 'data analytics manager',
       'business data analyst', 'applied data scientist',
       'staff data analyst', 'etl engineer', 'data devops engineer',
       'head of data', 'data science manager', 'data manager',
       'machine learning researcher', 'big data engineer',
       'data specialist', 'lead data analyst', 'bi data engineer',
       'director of data science', 'machine learning scientist',
       'mlops engineer', 'ai scientist', 'autonomous vehicle technician',
       'applied machine learning sc

In [46]:
stop_words = ['specialist','lead','tech','manager','consultant','director','head','staff','principal','of','software',"devops"]
 
def clear_phrase(phrase:str, stop_words:list) -> str:
    temp = []
    for word in phrase.split(" "):
        if word not in stop_words:
            temp.append(word.strip())
    return " ".join(temp)

list_job_ds_eng_ = []
for phrase in list_job_ds_eng:
    new_phrase = clear_phrase(phrase,stop_words)
    list_job_ds_eng_.append(new_phrase)

set(list_job_ds_eng_).__len__()

65

 Выявляем с датасета (vacancies.csv) за период 08/23-11/23  название вакансий, относящихся к DS. Отобранные вакансии я буду использовать для выбора вакансий из боьшого датасета.

In [51]:
df_vac = pd.read_csv("./raw_dataset/Kaggle_dataset/vacancies.csv", sep=",") 
df_vac.head(), df_vac.shape

(                                                name  \
 0                       Ведущий менеджер по продажам   
 1                    Data Scientist (команда Поиска)   
 2                   Data Scientist в b2c направление   
 3  Портфельный риск-аналитик / Middle Data Scient...   
 4                                    Трафик-менеджер   
 
                                                 tags  \
 0  ['Активные продажи', 'Ведение переговоров', 'Т...   
 1  ['Python', 'PyTorch', 'SQL', 'Machine Learning...   
 2  ['Python', 'Machine Learning', 'SQL', 'PyTorch...   
 3                                                 []   
 4  ['Анализ трафика', 'Аналитика', 'Маркетинг', '...   
 
                                                 link  \
 0  https://adsrv.hh.ru/click?b=533104&c=7&place=3...   
 1  https://saratov.hh.ru/vacancy/83116089?from=va...   
 2  https://saratov.hh.ru/vacancy/84009554?from=va...   
 3  https://saratov.hh.ru/vacancy/82421077?from=va...   
 4  https://saratov.hh.ru/v

Удаляем уточнения, содержащиеся в скобках к названием работы

In [143]:
df_vac.name = df_vac.name.str.lower()
df_vac.name = df_vac.name.str.replace("(\()[^()]+(\))", '', regex=True)
df_vac.name = df_vac.name.str.replace("(\,)[^()]+($)", '', regex=True)
job_vac_1 = df_vac.name.str.replace("(\[)[^()]+(\])", '', regex=True)
job_vac_1.nunique()

608

In [144]:
set(job_vac_1)

{'.net разработчик ',
 'account strategist / аккаунт-стратег / customer success director ',
 'analyst ',
 'analyst/data science',
 'appsec/data platform security bp',
 'aналитик качества данных',
 'backend developer php',
 'backend разработчик ',
 'bank senior accountant',
 'bi analyst ',
 'bi data engineer',
 'bi developer ',
 'bi developer & data analyst',
 'bi developer / data analyst',
 'bioinformatician - precision oncology ',
 'business analyst',
 'business data engineer',
 'business data engineer ',
 'business development manager',
 'business development manager / менеджер по развитию бизнеса ',
 'buyer / category manager',
 'chief data officer',
 'chief data officer ',
 'chief data officer / head of data',
 'chief marketing manager/head of marketing/cmo at crypto/blockchain/web3tech',
 'cis marketing manager for fintech',
 'claims negotiator specialist / ocean freight tanker cargo specialist',
 'compliance officer',
 'computer vision engineer',
 'cost accountant',
 'crm manager

In [146]:
enclude_words = ["data","scientist","analyst","analytics","аналитик","данных",'дата',"сайентист","маркетолог","маркетинга","цифрового","продуктовый",
                 'данными','моделей','модели','модель',"ai","science","аналитике","аналитики","аналитиков","аналитика","bi","ml","mle","анализа","анализу","анализ","dwh",
                 "1с",'osint',"исследователь","исследованиям","исследованию","исследований","прогнозированию", "прогнозирование","digital","cv","нейронаука","etl","поиску",
                 "машинного","машинное","машинному","нейросетевых","нейросетевого","нейронным","нейронных","зрения","modelling","intelligence","learning"]
                
vac_temp_ = []
bad_vac_ = []
for job in job_vac_1:
    if isinstance(job, str):
        job_list_ = re.split('[\\\/|]+',job.strip(' .?,!:;!@#$%^&*()-_+><[]{}\|/"'),flags=re.I)
        job_list = [' '.join(job.replace("-"," ").split()) for job in job_list_]
        #print(job_list)
        for j in job_list:
            if len(j) !=0:
                if set(enclude_words).intersection(set(re.split('[\s]+',j.strip(),flags=re.I))):
                    vac_temp_.append(j.strip())
                else:
                    bad_vac_.append(j.strip())
    else:
        print(job)

set(vac_temp_).__len__()
#set(bad_vac_)

nan
nan
nan


258

In [147]:
set(vac_temp_)

{'analyst',
 'aналитик качества данных',
 'bi analyst',
 'bi data engineer',
 'bi developer',
 'bi developer & data analyst',
 'big data',
 'business analyst',
 'business data engineer',
 'chief data officer',
 'chief digital marketing',
 'data',
 'data analysis specialist for ai model configuration',
 'data analyst',
 'data analyst billing',
 'data analyst middle',
 'data architect',
 'data engineer',
 'data engineer + bi',
 'data engineer ai',
 'data engineer developer',
 'data engineer dwh',
 'data engineer etl',
 'data engineer hadoop',
 'data engineer pyspark',
 'data engineer в команду business intelligence',
 'data engineer облако данных стрим источник данных',
 'data engineering team lead',
 'data lead analyst',
 'data loss prevention specialist remote',
 'data platform security bp',
 'data privacy manager',
 'data protection officer',
 'data quality engineer',
 'data science',
 'data science specialist',
 'data science team leader generative ai',
 'data scientict',
 'data scie

 Выявляем с датасета (Data Analyst Vacancies.csvм) название вакансий

In [149]:
df_dav = pd.read_csv("./raw_dataset/Kaggle_dataset/Data Analyst Vacancies.csv", sep=",")
df_dav.head()

,id,premium,name,department,has_test,response_letter_required,area,salary,type,address,...,working_time_intervals,working_time_modes,accept_temporary,professional_roles,accept_incomplete_resumes,experience,employment,adv_response_url,is_adv_vacancy,branding
0,86571284,False,"Data Scientist (стажер), удаленно",NaN,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...","{'from': None, 'to': 15000, 'currency': 'RUR',...","{'id': 'open', 'name': 'Открытая'}",NaN,...,"[{'id': 'from_four_to_six_hours_in_a_day', 'na...","[{'id': 'start_after_sixteen', 'name': 'Можно ...",False,"[{'id': '96', 'name': 'Программист, разработчи...",False,"{'id': 'noExperience', 'name': 'Нет опыта'}","{'id': 'probation', 'name': 'Стажировка'}",NaN,False,NaN
1,87273897,False,Data Analyst,NaN,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",NaN,"{'id': 'open', 'name': 'Открытая'}",NaN,...,[],[],False,"[{'id': '156', 'name': 'BI-аналитик, аналитик ...",False,"{'id': 'between3And6', 'name': 'От 3 до 6 лет'}","{'id': 'full', 'name': 'Полная занятость'}",NaN,False,NaN
2,87271975,False,Political analyst,NaN,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",NaN,"{'id': 'anonymous', 'name': 'Анонимная'}",NaN,...,[],[],False,"[{'id': '40', 'name': 'Другое'}]",False,"{'id': 'moreThan6', 'name': 'Более 6 лет'}","{'id': 'full', 'name': 'Полная занятость'}",NaN,False,NaN
3,86195321,False,Junior Data Scientist,NaN,False,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",NaN,"{'id': 'open', 'name': 'Открытая'}",NaN,...,[],[],False,"[{'id': '165', 'name': 'Дата-сайентист'}]",False,"{'id': 'between1And3', 'name': 'От 1 года до 3...","{'id': 'full', 'name': 'Полная занятость'}",NaN,False,"{'type': 'MAKEUP', 'tariff': None}"
4,87265216,False,Юрист,NaN,True,False,"{'id': '1', 'name': 'Москва', 'url': 'https://...",NaN,"{'id': 'open', 'name': 'Открытая'}",NaN,...,[],[],False,"[{'id': '146', 'name': 'Юрист'}]",False,"{'id': 'between1And3', 'name': 'От 1 года до 3...","{'id': 'full', 'name': 'Полная занятость'}",NaN,False,NaN


In [150]:
df_dav.shape

(1000, 36)

In [151]:
df_dav.name.nunique()

845

In [152]:
df_dav['name'] = df_dav.name.str.lower()
df_dav.name.nunique(), df_dav.name.unique()

(822,
 array(['data scientist (стажер), удаленно', 'data analyst',
        'political analyst', 'junior data scientist', 'юрист',
        'стажер (информационные технологии)',
        'ручной тестировщик по / qa manual (remote)', 'аналитик данных',
        'финансовый директор', 'junior data analyst',
        'аналитик данных / data scientist, удаленно', 'младший аналитик',
        'маркетинговый аналитик', 'стажер (data analyst)',
        'ведущий аналитик', 'data engineer',
        'аналитик данных (data analyst)',
        'big data analyst/аналитик больших данных',
        'специалист по поиску информации', 'директор по сервису',
        'младший аналитик / junior data analyst',
        'экономист-аналитик (junior data scientist)',
        'lead data analyst / ведущий аналитик данных',
        'дизайнер презентаций', 'junior game analyst (удаленно)',
        'qa-engineer/тестировщик (удалённо)',
        'стажер в направление развития новых продуктов',
        'младший веб-аналитик',

In [153]:
df_dav.name = df_dav.name.str.replace("(\()[^()]+(\))", '', regex=True)
df_dav.name = df_dav.name.str.replace("(\,)[^()]+($)", '', regex=True)
job_vac_ds = df_dav.name.str.replace("(\[)[^()]+(\])", '', regex=True)
vac_ds=job_vac_ds.unique()

объединяем все названия вакансий и по ним осуществляем отбор из большого датасета

In [159]:
len(vac_temp_.copy()), len(vac_temp.copy()), len(list_job_ds_eng.tolist().copy())

(542, 517, 93)

In [160]:
temp = vac_temp_.copy()
temp.extend(vac_temp.copy())
all_name_job = list_job_ds_eng.tolist().copy()
all_name_job.extend(temp)
set(all_name_job).__len__()

591

In [175]:
all_job = list(set(all_name_job))
temp = []
for job in all_job:
    new_job = re.sub(r"intern\D?|junior\D?|jr\D?|middle\D?|senior\D?","",job)
    temp.append(new_job)
list_uniq_jobs = list(set(temp))
list_uniq_jobs.__len__()

563

Подготовил регулярное выражение из названий должностей. По данной регулярке буду отбирать вакансии из большого датасета.

In [176]:
job_re = ["[\s-]".join(job.split()) for job in list_uniq_jobs]
reg_job= "|".join(job_re)
print(reg_job)

руководитель[\s-]направления[\s-]разработки[\s-]dwh|продуктовый[\s-]аналитик[\s-]в[\s-]карты|project[\s-]manager[\s-]на[\s-]продукт[\s-]мтс[\s-]аналитика|master[\s-]data[\s-]&[\s-]reporting[\s-]specialist|руководитель[\s-]направления[\s-]по[\s-]работе[\s-]с[\s-]данными|младший[\s-]инженер[\s-]по[\s-]визуализации[\s-]данных|machine[\s-]learning[\s-]developer|data[\s-]engineering[\s-]team[\s-]lead|bi[\s-]аналитик|big[\s-]data[\s-]architect|network[\s-]modelling[\s-]specialist|системный[\s-]аналитик[\s-]nrt|администратор[\s-]баз[\s-]данных|data[\s-]strategist|applied[\s-]machine[\s-]learning[\s-]scientist|администратор[\s-]big[\s-]data[\s-]сервисов|руководитель[\s-]группы[\s-]аналитиков|ibd[\s-]analyst|business[\s-]data[\s-]engineer|аналитик[\s-]больших[\s-]данных|lead[\s-]data[\s-]analyst|архитектор[\s-]dwh|количественный[\s-]аналитик|data[\s-]analyst[\s-]в[\s-]data[\s-]accelerator|эксперт[\s-]по[\s-]управлению[\s-]данными|head[\s-]of[\s-]digital[\s-]analytics|chief[\s-]digital[\s-]marke

Прововдим анализ столбцов, содержащих числовые значения

In [177]:
col_num = df.select_dtypes(include=np.number).columns
col_num.__len__(),col_num

(21,
 Index(['additional_premium', 'code_profession', 'foreign_workers_capability',
        'oknpo_code', 'okso_code', 'publication_period', 'regionNameTerm',
        'required_experience', 'retraining_condition', 'retraining_grant_value',
        'salary_min', 'salary_max', 'state_region_code', 'vacancy_address_code',
        'vacancy_address_latitude', 'vacancy_address_longitude', 'work_places',
        'federalDistrictCode', 'contactList', 'company_name', 'company_inn'],
       dtype='object'))

In [178]:
df.salary_min.nunique()

25692

Необходимо сократить количество столбцов, поэтому проводим анализ столбцов содержащих булевы значения

In [179]:
col_bool = df.select_dtypes(include='bool').columns
col_bool.__len__(), col_bool

(6,
 Index(['accommodation_capability', 'career_perspective', 'deleted',
        'is_moderated', 'is_uzbekistan_recruitment', 'is_quoted'],
       dtype='object'))

In [180]:
df[col_bool].nunique()

accommodation_capability     2
career_perspective           2
deleted                      1
is_moderated                 1
is_uzbekistan_recruitment    2
is_quoted                    2
dtype: int64

In [181]:
df.accommodation_capability.unique()

array([False,  True])

Проверяем актуальность вакансий по названиям колонок: is_moderated, deleted.

In [182]:
df.is_moderated.sum(), df.deleted.sum()

(603969, 0)

Вывод: все вакансии в датасете проверены и актуальны (т.е. нет удаленных).
Далее колонки, содержащие булевы значения, будут удалены из итогового датасета, так как сведения содержащиеся в данных колонках (возможность размещения, перспектива карьерного роста, статус актуальности, квотированность места) не представляют интерес для целевой задачи.

In [183]:
df.columns

Index(['id', 'academic_degree', 'accommodation_capability',
       'accommodation_type', 'additional_premium', 'additional_requirements',
       'bonus_type', 'measure_type', 'busy_type', 'career_perspective',
       'change_time', 'code_external_system', 'code_profession',
       'code_professional_sphere', 'company_code', 'contact_person',
       'contact_source', 'data_ids', 'date_create', 'date_modify', 'deleted',
       'education', 'education_speciality', 'foreign_workers_capability',
       'metro_ids', 'is_mobility_program', 'is_moderated',
       'is_uzbekistan_recruitment', 'is_quoted', 'need_medcard', 'oknpo_code',
       'okso_code', 'original_source_type', 'other_vacancy_benefit',
       'position_requirements', 'position_responsibilities',
       'publication_period', 'published_date', 'regionName', 'regionNameTerm',
       'company_business_size', 'required_certificates',
       'required_drive_license', 'required_experience',
       'retraining_capability', 'retraining_

In [184]:
dell_col = [
    "academic_degree", "bonus_type", "measure_type", "code_external_system",
    "contact_source","is_mobility_program","need_medcard","original_source_type","retraining_capability",
    "retraining_grant","source_type","status","transport_compensation","visibility","industryBranchName",
    "foreign_workers_capability","accommodation_capability","accommodation_type","career_perspective",
    "deleted","foreign_workers_capability","is_moderated","is_uzbekistan_recruitment","is_quoted","oknpo_code",
    "publication_period","regionNameTerm","retraining_condition","contactList","company_name","metro_ids",
    "okso_code","code_profession","company_code","contact_person","data_ids",'change_time','date_create',"published_date",
    "retraining_grant_value","required_drive_license","vacancy_address_additional_info","vacancy_address_house",
    "vacancy_address_latitude","vacancy_address_longitude","work_places","id","company_inn"
    
]
df = df.drop(dell_col, axis=1)
col_obj_new = df.select_dtypes(include=['object']).columns

In [185]:
df_ds = df[col_obj_new].apply(lambda col: col.str.contains(reg_job, na=False, flags=re.IGNORECASE, regex=True), axis=1)
df_ds

,additional_requirements,busy_type,code_professional_sphere,education,education_speciality,other_vacancy_benefit,position_requirements,position_responsibilities,regionName,company_business_size,...,social_protected_ids,vacancy_address,vacancy_benefit_ids,vacancy_name,professionalSphereName,full_company_name,company,languageKnowledge,hardSkills,softSkills
0,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
1,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
2,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
3,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
4,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627834,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
627835,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
627836,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False
627837,False,False,False,False,False,False,False,False,False,False,...,False,False,False,False,False,False,False,False,False,False


In [186]:
df_ds.sum()

additional_requirements        335
busy_type                        0
code_professional_sphere         0
education                        0
education_speciality            26
other_vacancy_benefit           68
position_requirements         1521
position_responsibilities     1848
regionName                       0
company_business_size        31078
required_certificates            8
salary                           0
schedule_type                    0
social_protected_ids             0
vacancy_address                  0
vacancy_benefit_ids              0
vacancy_name                  1254
professionalSphereName           0
full_company_name               58
company                      10245
languageKnowledge                0
hardSkills                      42
softSkills                       0
dtype: int64

In [187]:
new = df[df_ds.any(axis="columns")]
new

,additional_premium,additional_requirements,busy_type,code_professional_sphere,date_modify,education,education_speciality,other_vacancy_benefit,position_requirements,position_responsibilities,...,vacancy_address_code,vacancy_benefit_ids,vacancy_name,professionalSphereName,federalDistrictCode,full_company_name,company,languageKnowledge,hardSkills,softSkills
20,NaN,"<p style=""margin: 0"">возможно дополнительное п...",Полная занятость,Transport,2024-07-25 10:33:38+00:00,Не указано,NaN,NaN,"<p style=""margin: 0"">• опыт работы в розничной...","<p style=""margin: 0"">• предоставление услуг по...",...,5.800000e+16,NaN,Контролер-кассир,"Транспорт, автобизнес, логистика, склад, ВЭД",4.0,"ОТКРЫТОЕ АКЦИОНЕРНОЕ ОБЩЕСТВО ""РОССИЙСКИЕ ЖЕЛЕ...","{""companycode"":""1037739877295"",""email"":""Beloze...",[],[],[]
37,NaN,NaN,Временная,NotQualification,2024-07-25 01:17:16+00:00,Не указано,NaN,NaN,<p>Добросовестное исполнение должностных обяза...,"<p>Озеленение территории, закладка насаждений,...",...,2.400001e+16,NaN,рабочий по благоустройству и озеленению,"Работы, не требующие квалификации",6.0,"МУНИЦИПАЛЬНОЕ БЮДЖЕТНОЕ УЧРЕЖДЕНИЕ ""МНОГОПРОФИ...","{""companycode"":""1092443000275"",""email"":""Sibir_...",[],[],[]
57,NaN,NaN,Полная занятость,Education,2024-07-25 09:44:54+00:00,Среднее профессиональное,Экономика и бухгалтерский учет (по отраслям),<p>Стимулирующие выплаты и премии по итогам ра...,<ul> <li>Грамотная речь</li> <li>Коммуникабель...,<p>✅ преподавание теоретических и практических...,...,5.000006e+16,NaN,"Преподаватель ""Экономика и бухгалтерский учет""","Образование, наука",1.0,ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ПРОФЕССИОНАЛЬНОЕ ОБР...,"{""companycode"":""1045010203314"",""email"":""mo_che...",[],[],[]
71,NaN,NaN,Полная занятость,Education,2024-07-25 11:13:18+00:00,Высшее,NaN,NaN,<p>высшее профильное образование или переподго...,<ul> <li>Организация учебной деятельности обуч...,...,7.800000e+16,NaN,Преподаватель электротехники,"Образование, наука",2.0,Академия транспортных технологий,"{""companycode"":""8870c670-73f7-11ea-8b07-736ab1...",[],"[{""hard_skill_name"":""преподавание"",""owner_id"":...",[]
120,NaN,NaN,Полная занятость,Education,2024-07-25 06:05:09+00:00,Высшее,NaN,<p>Социальный пакет. Пояснение к режиму работы...,<p>Профессиональное педагогическое образование...,<p>Ответственное выполнение своих обязанностей...,...,6.600003e+16,NaN,Учитель музыки,"Образование, наука",5.0,МУНИЦИПАЛЬНОЕ АВТОНОМНОЕ ОБЩЕОБРАЗОВАТЕЛЬНОЕ У...,"{""companycode"":""1026600882660"",""email"":""school...",[],[],[]
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
627790,NaN,NaN,Полная занятость,Industry,2024-08-20 07:24:02+00:00,Не указано,NaN,NaN,"<p>опыт работы , водителькое удостоверение кат...",<p>2.1. Управление автопогрузчиком и&nbsp;всем...,...,3.601700e+16,NaN,Механизатор телескопического погрузчика,Производство,1.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ВОРО...","{""companycode"":""1173668014145"",""email"":""gb@vch...",[],[],[]
627799,NaN,<p>Заработная плата в соответствии с тарификац...,Полная занятость,Education,2024-08-23 11:18:02+00:00,Высшее,NaN,NaN,<p>Высшее профессиональное образование или сре...,<p>Организация учебной деятельности обучающихс...,...,7.800000e+16,NaN,преподаватель русского языка,"Образование, наука",2.0,САНКТ-ПЕТЕРБУРГСКОЕ ГОСУДАРСТВЕННОЕ БЮДЖЕТНОЕ ...,"{""companycode"":""1037816042538"",""email"":""pu35sp...",[],[],[]
627823,NaN,NaN,Полная занятость,Communal,2024-08-23 05:08:01+00:00,Среднее профессиональное,NaN,NaN,<p>Средне специальное или Высшее образование; ...,<p>Обеспечивает соблюдение технических норм эк...,...,8.600000e+16,NaN,инженер по эксплуатации жилых зданий,"ЖКХ, эксплуатация",5.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""СЕВЕР""","{""companycode"":""1058603056258"",""email"":""sibir_...",[],[],[]
627824,NaN,NaN,Полная занятость,Communal,2024-08-23 09:20:43+00:00,Не указано,NaN,NaN,<p>&nbsp;по результатам собеседования с работо...,<p>Управление грузовым автомобилем.&n

In [155]:
drop_coll = ["additional_premium","additional_requirements","education_speciality","other_vacancy_benefit","required_certificates","social_protected_ids","vacancy_benefit_ids"]
df_salary = df_salary.drop(drop_coll, axis=1)


In [212]:
new_vacancy_name.columns

Index(['additional_premium', 'additional_requirements', 'busy_type',
       'code_professional_sphere', 'date_modify', 'education',
       'education_speciality', 'other_vacancy_benefit',
       'position_requirements', 'position_responsibilities', 'regionName',
       'company_business_size', 'required_certificates', 'required_experience',
       'salary', 'salary_min', 'salary_max', 'schedule_type',
       'social_protected_ids', 'state_region_code', 'vacancy_address',
       'vacancy_address_code', 'vacancy_benefit_ids', 'vacancy_name',
       'professionalSphereName', 'federalDistrictCode', 'full_company_name',
       'company', 'languageKnowledge', 'hardSkills', 'softSkills'],
      dtype='object')

In [202]:
new_vacancy_name.vacancy_name.unique()

array(['Химик - аналитик', 'Программист 1С',
       'Разработчик 1С (middle, обычные формы)',
       'Консультант программист 1С', 'Разработчик 1С',
       'Системный аналитик', 'маркетолог', 'Маркетолог', 'Аналитик',
       'Аналитик в Департамент маркетинга', 'программист 1С',
       'Web разработчик Bitrix', 'Маркетолог по продукту',
       'Провизор-аналитик аптеки', 'Младший консультант 1С',
       'Консультант 1С', 'Старший системный аналитик',
       'Химик аналитик лаборант',
       'Инженер отдела мониторинга и аналитики ПО "Северные электрические сети"',
       'аналитик организационно-аналитической работы', 'Аналитик данных',
       'Ведущий специалист  отдела аналитики транспортной работы',
       'Аналитик МФЦ', 'администратор баз данных 1 категории',
       'Ведущий маркетолог', 'контролер (продуктовый павильон)',
       'Разработчик-программист 1С',
       'Специалист 2 категории (отдел аналитики)', 'трейд-маркетолог',
       'Аналитик I категории',
       'Аналитик (в о

Отчищаем вакансии отличных от специализации DS

In [234]:
new_vacancy_name_ = new_vacancy_name[["vacancy_name"]].apply(lambda col: col.str.contains("pазработчик|программист|химик|магазин|водитель|принтера|бухгалтер|доцент|call-центр", na=False, flags=re.IGNORECASE, regex=True), axis =1)

In [235]:
new_vacancy_name_.sum()

vacancy_name    298
dtype: int64

Осталось 956 вакансий подходящих нашим критериям

In [233]:
ds_vac_all = new_vacancy_name[~new_vacancy_name_.any(axis="columns")]
ds_vac_all

,additional_premium,additional_requirements,busy_type,code_professional_sphere,date_modify,education,education_speciality,other_vacancy_benefit,position_requirements,position_responsibilities,...,vacancy_address_code,vacancy_benefit_ids,vacancy_name,professionalSphereName,federalDistrictCode,full_company_name,company,languageKnowledge,hardSkills,softSkills
3514,NaN,NaN,Полная занятость,InformationTechnology,2024-07-26 06:04:59+00:00,Среднее профессиональное,Информатика,NaN,<ul> <li>уверенные знания механизмов платформы...,<ul> <li>разработка нового функционала и корре...,...,4.500000e+16,ДМС,"Разработчик 1С (middle, обычные формы)","Информационные технологии, телекоммуникации, с...",5.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ВОСТ...","{""companycode"":""1227700184161"",""email"":""info@v...",[],[],[]
4287,NaN,NaN,Полная занятость,InformationTechnology,2024-07-26 06:02:38+00:00,Высшее,Информационные технологии,NaN,<ul> <li>уверенные знания механизмов платформы...,<ul> <li>разработка новой корпоративной систем...,...,6.600000e+16,ДМС,Разработчик 1С,"Информационные технологии, телекоммуникации, с...",5.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ВОСТ...","{""companycode"":""1227700184161"",""email"":""info@v...",[],[],[]
4459,NaN,NaN,Полная занятость,InformationTechnology,2024-07-26 10:57:10+00:00,Среднее профессиональное,NaN,NaN,<ul> <li>Опыт работы в качестве ИТ-аналитика н...,<ul> <li>Активное участие в коммуникациях с за...,...,5.900000e+16,NaN,Системный аналитик,"Информационные технологии, телекоммуникации, с...",4.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ПРОГ...","{""companycode"":""1165958087228"",""email"":""info@p...",[],[],[]
5311,NaN,NaN,Полная занятость,Transport,2024-07-25 23:02:41+00:00,Высшее,NaN,NaN,<p>опыт работы в маркетинге</p>,<p>&bull; Создание маркетинговой стратегии;&bu...,...,2.500000e+16,NaN,маркетолог,"Транспорт, автобизнес, логистика, склад, ВЭД",7.0,"ООО "" Альтаир-Авто""","{""companycode"":""9fdc1f30-7578-11ea-bdff-736ab1...",[],[],[]
7585,NaN,NaN,Полная занятость,Industry,2024-07-25 05:47:16+00:00,Высшее,Маркетинг,NaN,<ul> <li>высшее образование</li> <li>креативно...,<ul> <li>лидогенерация</li> <li>исследование р...,...,2.200000e+16,NaN,Маркетолог,Производство,6.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""НИТР...","{""companycode"":""1202200000650"",""email"":""697014...",[],"[{""hard_skill_name"":""умение работать в условия...","[{""owner_id"":""dd3835c5-40b9-11ee-ad0a-9586bb63..."
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
625323,NaN,<p><b>Ведущий аналитик отдела по правовому соп...,Полная занятость,StateServices,2024-08-23 11:18:05+00:00,Высшее,NaN,"<p>Ежемесячная премия, квартальная премия, год...",<p><!-- [if !supportLists]-->-&nbsp;&nbsp;&nbs...,<p>- проведение юридической экспертизы на пред...,...,5.900000e+16,NaN,Ведущий аналитик,"Государственная служба, некоммерческие организ...",4.0,АДМИНИСТРАЦИЯ ГОРОДА ПЕРМИ,"{""companycode"":""1025900532460"",""email"":""gosusl...",[],[],[]
625356,NaN,<br/><p><b>Условия:</b><p></p><ul><li>Возможно...,Полная занятость,AccountingTaxesManagement,2024-08-23 05:05:03+00:00,Высшее,NaN,NaN,<p></p><li>Образование: высшее экономическое/ф...,</p><p></p><li>Обеспечение единой стратегии ра...,...,7.700000e+16,NaN,Специалист по учету и аналитике,"Бухгалтерия, налоги, управленческий учет",1.0,Пензанефтепродукт,"{""companycode"":""1027700404797"",""hr-agency"":fal...",[],[],[]
625482,100.0,NaN,Полная занятость,Communal,2024-08-23 06:03:08+00:00,Высшее,Экономика,NaN,NaN,<p>- Формирование предложений для включения в ...,...,2.600000e+16,NaN,Аналитик отдела планирования и анализа,"ЖКХ, эксплуатация",8.0,КОМИТЕТ ГОРОДСКОГО ХОЗЯЙСТВА АДМИНИСТРАЦИИ ГОР...,"{""companycode"":""1052600306770"",""email"":""kgh@st...",[],[],[]
625540,NaN,NaN,Полная занятость,ServiceMaintenance,2024-08-13 06:22:22+00:00,Высшее,Автоматизированные системы обработки и...,NaN,"<p>Коммуникабельность, дисциплинированность</p>",<p>- осуществляет п

In [236]:
ds_vac_all.info()

<class 'pandas.core.frame.DataFrame'>
Index: 956 entries, 3514 to 626695
Data columns (total 31 columns):
 #   Column                     Non-Null Count  Dtype              
---  ------                     --------------  -----              
 0   additional_premium         116 non-null    float64            
 1   additional_requirements    300 non-null    object             
 2   busy_type                  956 non-null    object             
 3   code_professional_sphere   956 non-null    object             
 4   date_modify                956 non-null    datetime64[ns, UTC]
 5   education                  956 non-null    object             
 6   education_speciality       235 non-null    object             
 7   other_vacancy_benefit      148 non-null    object             
 8   position_requirements      893 non-null    object             
 9   position_responsibilities  945 non-null    object             
 10  regionName                 941 non-null    object             
 11  compa

In [26]:
ds_vac_all.salary

0      от 150000
1      от 180000
2       от 60000
3       от 60000
4       от 70000
         ...    
951     от 30000
952     от 50000
953     от 35000
954     от 22129
955     от 21000
Name: salary, Length: 956, dtype: object

Исследую содержане колонок

additional_requirements - извлечение скиллов  
position_requirements - извлечение скиллов  
position_responsibilities - извлечение скиллов  
"code_professional_sphere" - not qualif  ====== professionalSphereName on RUS  
required_experience - yes nan

In [ ]:
col_needs = ["vacancy_name","additional_requirements","position_requirements","position_responsibilities",
             "busy_type","code_professional_sphere","education","regionName","","company_business_size",
             "required_experience","salary","salary_min","salary_max","schedule_type",
             "vacancy_address","vacancy_address_code","federalDistrictCode"]

In [277]:
col = "hardSkills"
(ds_vac_all[col].isnull().sum(),
ds_vac_all[col].nunique(),
ds_vac_all[col].unique())

(0,
 151,
 array(['[]',
        '[{"hard_skill_name":"умение работать в условиях многозадачности","owner_id":"dd3835c5-40b9-11ee-ad0a-9586bb63c653","type":"HardSkill"},{"hard_skill_name":"аналитические способности","owner_id":"dd3835c5-40b9-11ee-ad0a-9586bb63c653","type":"HardSkill"}]',
        '[{"hard_skill_name":"владение компьютером на уровне опытного пользователя","owner_id":"cfe833b0-79ef-11ee-90c9-e7d0d2cf29b1","type":"HardSkill"},{"hard_skill_name":"знание компьютерных программ","owner_id":"cfe833b0-79ef-11ee-90c9-e7d0d2cf29b1","type":"HardSkill"}]',
        '[{"hard_skill_name":"jira","owner_id":"d63394e5-a78f-11ed-ba16-e3f7ccb48a0a","type":"HardSkill"}]',
        '[{"hard_skill_name":"коммуникабельность","owner_id":"7ce48e78-4d84-11ef-bdb0-d549be31d974","type":"HardSkill"}]',
        '[{"hard_skill_name":"ведения бухгалтерского и налогового учета и отчетности","owner_id":"ee4b1312-7537-11ea-a8d5-bf2cfe8c828d","type":"HardSkill"},{"hard_skill_name":"1с","owner_id":"ee4b1312-75

In [276]:
col = "softSkills"
(ds_vac_all[col].isnull().sum(),
ds_vac_all[col].nunique(),
ds_vac_all[col].unique())

(0,
 67,
 array(['[]',
        '[{"owner_id":"dd3835c5-40b9-11ee-ad0a-9586bb63c653","soft_skill_name":"Коммуникабельность","type":"SoftSkill"},{"owner_id":"dd3835c5-40b9-11ee-ad0a-9586bb63c653","soft_skill_name":"Креативность","type":"SoftSkill"},{"owner_id":"dd3835c5-40b9-11ee-ad0a-9586bb63c653","soft_skill_name":"Стрессоустойчивость","type":"SoftSkill"}]',
        '[{"owner_id":"ee4b1312-7537-11ea-a8d5-bf2cfe8c828d","soft_skill_name":"Коммуникабельность","type":"SoftSkill"},{"owner_id":"ee4b1312-7537-11ea-a8d5-bf2cfe8c828d","soft_skill_name":"Быстрая обучаемость","type":"SoftSkill"},{"owner_id":"ee4b1312-7537-11ea-a8d5-bf2cfe8c828d","soft_skill_name":"Способность обучать других","type":"SoftSkill"}]',
        '[{"owner_id":"63ef85e9-a7be-11ed-b894-05dc90903fb8","soft_skill_name":"Коммуникабельность","type":"SoftSkill"},{"owner_id":"63ef85e9-a7be-11ed-b894-05dc90903fb8","soft_skill_name":"Ответственность","type":"SoftSkill"},{"owner_id":"63ef85e9-a7be-11ed-b894-05dc90903fb8","soft_ski

In [269]:
col = "company"
(ds_vac_all[col].isnull().sum(),
ds_vac_all[col].nunique(),
ds_vac_all[col].unique())

(0,
 691,
 array(['{"companycode":"1227700184161","email":"info@vostok-electra.ru","hr-agency":false,"inn":"7708403610","kpp":"770801001","name":"ООО \\"ВОСТОК.СОФТ\\"","ogrn":"1227700184161","phone":"+7(495)7752497","url":"https://trudvsem.ru/company/1227700184161"}',
        '{"companycode":"1165958087228","email":"info@prg-it.ru","hr-agency":false,"inn":"5902037255","kpp":"784001001","name":"ООО \\"ПИТ\\"","ogrn":"1165958087228","phone":"+7(342)2563151","site":"https://simpl-group.ru/","url":"https://trudvsem.ru/company/1165958087228"}',
        '{"companycode":"9fdc1f30-7578-11ea-bdff-736ab11edb0c","hr-agency":false,"inn":"2538098830","kpp":"250201001","name":"ООО \\"Альтаир-Авто\\"","ogrn":"1062538056624","url":"https://trudvsem.ru/company/9fdc1f30-7578-11ea-bdff-736ab11edb0c"}',
        '{"companycode":"1202200000650","email":"697014@mail.ru","hr-agency":false,"inn":"2223631483","kpp":"222101001","name":"ООО \\"НИТРОГЕН\\"","ogrn":"1202200000650","phone":"+7(913)2221466","url":"h

In [267]:
col = "federalDistrictCode"
(ds_vac_all[col].isnull().sum(),
ds_vac_all[col].nunique(),
ds_vac_all[col].unique())

(0, 8, array([5., 4., 7., 6., 2., 1., 3., 8.]))

In [266]:
col = "professionalSphereName"
(ds_vac_all[col].isnull().sum(),
ds_vac_all[col].nunique(),
ds_vac_all[col].unique())

(0,
 33,
 array(['Информационные технологии, телекоммуникации, связь',
        'Транспорт, автобизнес, логистика, склад, ВЭД', 'Производство',
        'Банки, кредит, страхование, пенсионное обеспечение',
        'Маркетинг, реклама, PR',
        'Государственная служба, некоммерческие организации',
        'Образование, наука', 'Здравоохранение и социальное обеспечение',
        'Продажи, закупки, снабжение, торговля', 'Высший менеджмент',
        'Электроэнергетика', 'Безопасность, службы охраны',
        'Услуги населению, сервисное обслуживание',
        'Бухгалтерия, налоги, управленческий учет',
        'Консалтинг, стратегическое развитие, управление',
        'ЖКХ, эксплуатация',
        'Лесная, деревообрабатывающая, целлюлозно-бумажная промышленность',
        'Административная работа, секретариат, АХО',
        'Строительство, ремонт, стройматериалы, недвижимость',
        'Химическая, нефтехимическая, топливная промышленность',
        'Спорт, фитнес, салоны красоты', 'Добы

In [265]:
col = "vacancy_address_code"
(ds_vac_all[col].isnull().sum(),
ds_vac_all[col].nunique(),
ds_vac_all[col].unique())

(0,
 697,
 array([4.50000010e+16, 6.60000010e+16, 5.90000010e+16, 2.50000030e+16,
        2.20000010e+16, 7.80000000e+16, 7.70000000e+16, 3.40000060e+16,
        2.60000010e+16, 2.00000100e+15, 8.60000100e+16, 1.20000010e+16,
        2.70000000e+12, 4.10000010e+16, 5.00000250e+16, 7.70000000e+16,
        5.10000010e+16, 3.80000000e+12, 7.80000000e+16, 7.70000000e+16,
        6.60000010e+16, 5.20000010e+16, 7.80000000e+16, 7.20000010e+16,
        3.30000010e+16, 7.40000010e+16, 7.40000010e+16, 2.01100100e+15,
        2.50000010e+16, 6.60000010e+16, 2.30000010e+16, 8.90000070e+16,
        5.50000010e+16, 7.80000000e+16, 5.10000020e+16, 7.70000000e+16,
        7.60000010e+12, 2.30000070e+16, 2.00000100e+15, 3.80150000e+16,
        5.20160010e+16, 7.80000000e+16, 5.50250000e+16, 8.60000020e+16,
        8.60000100e+16, 4.30000010e+16, 9.10000070e+16, 2.20000010e+16,
        7.70000000e+12, 7.70000000e+16, 7.70000000e+16, 5.70000010e+16,
        5.90000010e+16, 6.30000010e+16, 9.10000070e+16

In [264]:
col = "vacancy_address"
(ds_vac_all[col].isnull().sum(),
ds_vac_all[col].nunique(),
ds_vac_all[col].unique())

(15,
 779,
 array(['Курганская область, Город Курган, Тимофея Невежина улица, 3',
        'Свердловская область, Город Екатеринбург, Чапаева улица, 3',
        'Пермский край, Город Пермь, Космонавтов шоссе, 111',
        'Приморский край, Город Артем, Тульская улица, 20',
        'Алтайский край, Город Барнаул, Заводской 9-й проезд, 56',
        'Город Санкт-Петербург, Академика Павлова улица',
        'Город Москва, Беловежская улица, дом: Д.4;',
        'Волгоградская область, Город Фролово, Донская улица, 7',
        'Ставропольский край, Город Ставрополь, Серова улица, ДОМ 241',
        'Республика Башкортостан, Город Уфа, Трамвайная улица, 2/4',
        'Ханты-Мансийский автономный округ - Югра, Город Сургут, Мира проспект, дом: Д. 43;',
        'Республика Марий Эл, Город Йошкар-Ола, Воскресенский проспект, 9',
        'Хабаровский край',
        'Камчатский край, Город Петропавловск-Камчатский, Ленинградская улица, 128',
        'Московская область, Город Видное, Мещерино посел

In [263]:
col = "state_region_code"
(ds_vac_all[col].isnull().sum(),
ds_vac_all[col].nunique(),
ds_vac_all[col].unique())

(0,
 78,
 array([4.5e+12, 6.6e+12, 5.9e+12, 2.5e+12, 2.2e+12, 7.8e+12, 7.7e+12,
        3.4e+12, 2.6e+12, 2.0e+11, 8.6e+12, 1.2e+12, 2.7e+12, 4.1e+12,
        5.0e+12, 5.1e+12, 3.8e+12, 5.2e+12, 7.2e+12, 3.3e+12, 7.4e+12,
        2.3e+12, 8.9e+12, 5.5e+12, 7.6e+12, 4.3e+12, 9.1e+12, 5.7e+12,
        6.3e+12, 6.1e+12, 3.2e+12, 1.6e+12, 3.6e+12, 7.1e+12, 6.4e+12,
        2.4e+12, 7.0e+11, 3.0e+12, 7.5e+12, 5.4e+12, 1.1e+12, 4.2e+12,
        1.8e+12, 3.1e+12, 5.3e+12, 4.8e+12, 5.8e+12, 1.0e+12, 6.8e+12,
        1.3e+12, 2.1e+12, 1.9e+12, 7.0e+12, 3.7e+12, 7.3e+12, 6.9e+12,
        4.0e+12, 3.5e+12, 6.2e+12, 4.4e+12, 2.9e+12, 6.5e+12, 4.7e+12,
        6.7e+12, 3.0e+11, 5.6e+12, 1.4e+12, 6.0e+12, 3.9e+12, 9.2e+12,
        1.0e+11, 9.4e+12, 1.5e+12, 9.3e+12, 7.9e+12, 8.0e+11, 4.9e+12,
        4.6e+12]))

In [262]:
col = "schedule_type"
(ds_vac_all[col].isnull().sum(),
ds_vac_all[col].nunique(),
ds_vac_all[col].unique())

(0,
 6,
 array(['Полный рабочий день', 'Гибкий график',
        'Ненормированный рабочий день', 'Неполный рабочий день',
        'Вахтовый метод', 'Сменный график'], dtype=object))

In [261]:
col = "salary_min"
(ds_vac_all[col].isnull().sum(),
ds_vac_all[col].nunique(),
ds_vac_all[col].unique())

(0,
 240,
 array([150000., 180000.,  60000.,  70000., 100000.,  50000.,  19242.,
         25000., 175000.,  45000.,  21114.,  55000.,  16630.,      0.,
         18057.,  35000., 130000.,  33960.,  57240.,  27000.,  40000.,
         32000.,  69000.,  30000.,  20000., 110000., 104500.,  44540.,
         90000.,  49612.,  29000.,  22129.,  54000.,  47165.,  41500.,
         23000., 170000.,  43000., 252000.,  43750.,  74000.,  71350.,
         22130.,  42332.,  84000.,  50029.,   9621.,  76000.,  27500.,
         51067.,  80000., 230000.,  37357.,  64600.,  19245.,  75000.,
         30708.,  34000.,  65000.,  66939., 120000.,  38000.,  39840.,
        167000.,  31800.,  28273.,  33000.,  92000.,  44000.,  36000.,
         41338.,  36500.,  44257.,  22000.,  46200.,  25015.,  30800.,
         34441.,  51000., 280000.,  60775., 104000., 140000.,  62250.,
         41000., 200000.,  44256.,   9622.,  20650.,  28000.,  26000.,
         32500.,  52800.,  22500.,  39000.,  22200.,  42000.,  5497

In [259]:
col = "required_experience"
(ds_vac_all[col].isnull().sum(),
ds_vac_all[col].nunique(),
ds_vac_all[col].unique())

(8, 10, array([ 3.,  5.,  2.,  4.,  1.,  0., nan, 10.,  7.,  6., 12.]))

In [258]:
col = "company_business_size"
(ds_vac_all[col].isnull().sum(),
ds_vac_all[col].nunique(),
ds_vac_all[col].unique())

(0, 5, array(['SMALL', 'MIDDLE', 'MICRO', 'LARGE', 'BIG'], dtype=object))

In [25]:
ds_vac_all["regionName"]

0        Курганская область
1      Свердловская область
2             Пермский край
3           Приморский край
4            Алтайский край
               ...         
951           Пермский край
952            Город Москва
953     Ставропольский край
954          Омская область
955     Ставропольский край
Name: regionName, Length: 956, dtype: object

In [257]:
col = "regionName"
(ds_vac_all[col].isnull().sum(),
ds_vac_all[col].nunique(),
ds_vac_all[col].unique())

(15,
 78,
 array(['Курганская область', 'Свердловская область', 'Пермский край',
        'Приморский край', 'Алтайский край', 'Город Санкт-Петербург',
        'Город Москва', 'Волгоградская область', 'Ставропольский край',
        'Республика Башкортостан',
        'Ханты-Мансийский автономный округ - Югра', 'Республика Марий Эл',
        'Хабаровский край', 'Камчатский край', 'Московская область',
        'Мурманская область', 'Иркутская область', 'Нижегородская область',
        'Тюменская область', 'Владимирская область', 'Челябинская область',
        'Краснодарский край', 'Ямало-Ненецкий автономный округ',
        'Омская область', 'Ярославская область', 'Кировская область',
        'Республика Крым', 'Орловская область', 'Самарская область',
        'Ростовская область', 'Брянская область',
        'Республика Татарстан (Татарстан)', 'Воронежская область',
        'Тульская область', 'Саратовская область', 'Красноярский край',
        'Кабардино-Балкарская Республика', 'Астраханс

In [253]:
col = "education"
(ds_vac_all[col].isnull().sum(),
ds_vac_all[col].nunique(),
ds_vac_all[col].unique())

(0,
 5,
 array(['Среднее профессиональное', 'Высшее', 'Не указано', 'Среднее',
        'Незаконченное высшее'], dtype=object))

In [252]:
col = "code_professional_sphere"
(ds_vac_all[col].isnull().sum(),
ds_vac_all[col].nunique(),
ds_vac_all[col].unique())

(0,
 33,
 array(['InformationTechnology', 'Transport', 'Industry', 'Finances',
        'Marketing', 'StateServices', 'Education', 'Medicine', 'Sales',
        'Management', 'ElectricpowerIndustry', 'Safety',
        'ServiceMaintenance', 'AccountingTaxesManagement', 'Consulting',
        'Communal', 'Forest', 'DeskWork', 'BuldindRealty',
        'ChemicalAndFuelIndustry', 'SportsFitnessBeautySalons',
        'Resources', 'MechanicalEngineering', 'HumanRecruitment', 'Food',
        'Agricultural', 'RootLightIndustry', 'Jurisprudence',
        'Restaurants', 'NotQualification', 'WorkingSpecialties', 'Culture',
        'Metallurgy'], dtype=object))

In [315]:
col = "code_professional_sphere"
(ds_vac_all[col]=="InformationTechnology").sum()

291

In [251]:
col = "busy_type"
(ds_vac_all[col].isnull().sum(),
ds_vac_all[col].nunique(),
ds_vac_all[col].unique())


(0,
 5,
 array(['Полная занятость', 'Удаленная', 'Временная',
        'Частичная занятость', 'Стажировка'], dtype=object))

In [312]:
col_needs = ["vacancy_name","additional_requirements","position_requirements","position_responsibilities",
             "busy_type","code_professional_sphere","education","regionName","","company_business_size",
             "required_experience","salary","salary_min","salary_max","schedule_type",
             "vacancy_address","vacancy_address_code","federalDistrictCode"]

In [317]:
ds_vac_all.to_csv("./salary_ds_vac_all.csv", sep ="|")

In [95]:
ds_vac_all = pd.read_csv("./salary_ds_vac_all.csv", sep ="|")
ds_vac_all.columns

Index(['Unnamed: 0', 'additional_premium', 'additional_requirements',
       'busy_type', 'code_professional_sphere', 'date_modify', 'education',
       'education_speciality', 'other_vacancy_benefit',
       'position_requirements', 'position_responsibilities', 'regionName',
       'company_business_size', 'required_certificates', 'required_experience',
       'salary', 'salary_min', 'salary_max', 'schedule_type',
       'social_protected_ids', 'state_region_code', 'vacancy_address',
       'vacancy_address_code', 'vacancy_benefit_ids', 'vacancy_name',
       'professionalSphereName', 'federalDistrictCode', 'full_company_name',
       'company', 'languageKnowledge', 'hardSkills', 'softSkills'],
      dtype='object')

In [96]:
ds_vac_all.shape

(956, 32)

Определяю скилы по вакансиям через регулярные выражения

In [97]:
def func(x):
    if isinstance(x,list):
        rez = list(set([i.strip() for i in x if len(i.strip())>1 and not i.replace(" ",'').isdigit()]))#i.strip().isdigit()
        if len(set([i.strip() for i in x if len(i.strip())>1 and not i.replace(" ",'').isdigit()]))==0:
            rez = np.NaN
    else:
        rez = np.NaN
    return rez

col_ = "additional_requirements"
ds_vac_all[col_] = ds_vac_all[col_].str.replace("(\<)[^()]+(\>)|ndash|laquo|raquo|nbsp|middot|bull|amp", '', regex=True)
ds_vac_all[f"skill_{col_}"] = ds_vac_all[col_].str.findall(r"\b[A-Za-z0-9 ]*\b", flags=re.I).apply(lambda x : func(x))
ds_vac_all[f"skill_{col_}"].value_counts()
#ds_vac_all[f"skill_{col_}"].isnull().sum()

skill_additional_requirements
[SMART]                                           2
[Postman]                                         1
[Python, Java]                                    1
[IT]                                              1
[Forbes, Vademecum]                               1
[BPMN]                                            1
[Word]                                            1
[b2b]                                             1
[New Balance, World Class, Pilates class, fit]    1
[Agile, Kanban, SCRUM]                            1
Name: count, dtype: int64

In [98]:
col_ = "position_requirements"
ds_vac_all[col_] = ds_vac_all[col_].str.replace("(\<)[^()]+(\>)|ndash|laquo|raquo|nbsp|middot|bull|amp", '', regex=True)
ds_vac_all[f"skill_{col_}"] = ds_vac_all[col_].str.findall(r"\b[A-Za-z0-9 ]*\b", flags=re.I).apply(lambda x : func(x))
ds_vac_all[f"skill_{col_}"].value_counts()
ds_vac_all[f"skill_{col_}"].isnull().sum()

732

In [99]:
col_ = "position_responsibilities"
ds_vac_all[col_] = ds_vac_all[col_].str.replace("(\<)[^()]+(\>)|ndash|laquo|raquo|nbsp|middot|bull|amp", '', regex=True)
ds_vac_all[f"skill_{col_}"] = ds_vac_all[col_].str.findall(r"\b[A-Za-z0-9 ]*\b", flags=re.I).apply(lambda x : func(x))
ds_vac_all[f"skill_{col_}"].value_counts()
ds_vac_all[f"skill_{col_}"].isnull().sum()

858

In [100]:
col_ = "hardSkills" #(?:")([^:]*)(?:")
ds_vac_all[f"skill_{col_}"] = ds_vac_all[col_].str.findall(
    r'(?:"hard_skill_name":")([^:]*)(?:","owner_id")',
    flags=re.I).apply(lambda x: [re.sub(r"\b[^A-Za-z0-9]*\b","", s, flags=re.I) for s in x if len(s)!=0]).apply(lambda x : func(x))
ds_vac_all[f"skill_{col_}"].value_counts()
ds_vac_all[f"skill_{col_}"].isnull().sum()

908

In [101]:
col_ = "softSkills" #(?:")([^:]*)(?:")
#ds_vac_all[f"skill_{col_}"] 
ttt= ds_vac_all[col_].str.findall(
    r'(?:"soft_skill_name":")([^:]*)(?:","type")',
    flags=re.I).apply(lambda x: [s for s in x if list(s)])
#ds_vac_all[f"skill_{col_}"].value_counts()
ttt.value_counts()

softSkills
[]                                                                                                                 890
[Ответственность, Дисциплинированность]                                                                              3
[Ответственность]                                                                                                    2
[Коммуникабельность, Работа в команде]                                                                               2
[Ответственность, Коммуникабельность, Работа в команде]                                                              2
                                                                                                                  ... 
[Умение достигать желаемых результатов, Инициативность, Креативность]                                                1
[Ответственность, Дисциплинированность, Исполнительность, Аккуратность]                                              1
[Настойчивость, Инициативность, Ответ

In [102]:
ds_vac_all.head()

,Unnamed: 0,additional_premium,additional_requirements,busy_type,code_professional_sphere,date_modify,education,education_speciality,other_vacancy_benefit,position_requirements,...,federalDistrictCode,full_company_name,company,languageKnowledge,hardSkills,softSkills,skill_additional_requirements,skill_position_requirements,skill_position_responsibilities,skill_hardSkills
0,3514,NaN,NaN,Полная занятость,InformationTechnology,2024-07-26 06:04:59+00:00,Среднее профессиональное,Информатика,NaN,,...,5.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ВОСТ...","{""companycode"":""1227700184161"",""email"":""info@v...",[],[],[],NaN,NaN,NaN,NaN
1,4287,NaN,NaN,Полная занятость,InformationTechnology,2024-07-26 06:02:38+00:00,Высшее,Информационные технологии,NaN,знание технологий обмена и опыт работы с механ...,...,5.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ВОСТ...","{""companycode"":""1227700184161"",""email"":""info@v...",[],[],[],NaN,"[http, web]",NaN,NaN
2,4459,NaN,NaN,Полная занятость,InformationTechnology,2024-07-26 10:57:10+00:00,Среднее профессиональное,NaN,NaN,,...,4.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""ПРОГ...","{""companycode"":""1165958087228"",""email"":""info@p...",[],[],[],NaN,NaN,NaN,NaN
3,5311,NaN,NaN,Полная занятость,Transport,2024-07-25 23:02:41+00:00,Высшее,NaN,NaN,,...,7.0,"ООО "" Альтаир-Авто""","{""companycode"":""9fdc1f30-7578-11ea-bdff-736ab1...",[],[],[],NaN,NaN,[BTL],NaN
4,7585,NaN,NaN,Полная занятость,Industry,2024-07-25 05:47:16+00:00,Высшее,Маркетинг,NaN,креативное мышление (интересные идеи и решения...,...,6.0,"ОБЩЕСТВО С ОГРАНИЧЕННОЙ ОТВЕТСТВЕННОСТЬЮ ""НИТР...","{""companycode"":""1202200000650"",""email"":""697014...",[],"[{""hard_skill_name"":""умение работать в условия...","[{""owner_id"":""dd3835c5-40b9-11ee-ad0a-9586bb63...",NaN,NaN,NaN,NaN


In [103]:
def join_row(r1, r2):
    if isinstance(r1,(float,type(None))) and not isinstance(r2,(float,type(None))):
        rez = r2
    elif not isinstance(r1,(float,type(None))) and isinstance(r2,(float,type(None))):
        rez = r1
    elif not isinstance(r1,(float,type(None))) and not isinstance(r2,(float,type(None))):
        rez = r1.extend(r2)
    elif isinstance(r1,(float,type(None))) and isinstance(r2,(float,type(None))):
        rez = np.NaN
    return rez

ds_vac_all['skill_additional_requirements'] = ds_vac_all.apply(
    lambda row: join_row(row["skill_additional_requirements"], row["skill_position_requirements"]),
    axis=1
)
ds_vac_all['skill_additional_requirements'] = ds_vac_all.apply(
    lambda row: join_row(row["skill_additional_requirements"], row["skill_position_responsibilities"]),
    axis=1
)

ds_vac_all['skill_additional_requirements'] = ds_vac_all.apply(
    lambda row: join_row(row["skill_additional_requirements"], row["skill_hardSkills"]),
    axis=1
)
ds_vac_all['skill_additional_requirements'].isnull().sum()

709

In [106]:
ds_vac_all['skills_all']=ds_vac_all['skill_additional_requirements']
ds_vac_all['skills_all'] =  ds_vac_all['skills_all'].apply(lambda x : ",".join(list(set(x))) if not isinstance(x,(float,type(None))) else np.nan)
#ds_vac_all['skills_all_'] = ds_vac_all['skills_all'].apply(lambda x : ",".join(list(set([i for i in x.split(",") if not isinstance(i,(float,type(None))) and len(i.split())<4]))))
ds_vac_all['skills_all']

0           NaN
1      http,web
2           NaN
3           BTL
4           NaN
         ...   
951         NaN
952         NaN
953         NaN
954         NaN
955         NaN
Name: skills_all, Length: 956, dtype: object

In [107]:
ds_vac_all['skills_all'].isnull().sum()

709

In [109]:
ds_vac_all.head()

,Unnamed: 0,additional_premium,additional_requirements,busy_type,code_professional_sphere,date_modify,education,education_speciality,other_vacancy_benefit,position_requirements,...,company,languageKnowledge,hardSkills,softSkills,skill_additional_requirements,skill_position_requirements,skill_position_responsibilities,skill_hardSkills,skills_all,skills_all_
0,3514,NaN,NaN,Полная занятость,InformationTechnology,2024-07-26 06:04:59+00:00,Среднее профессиональное,Информатика,NaN,,...,"{""companycode"":""1227700184161"",""email"":""info@v...",[],[],[],NaN,NaN,NaN,NaN,NaN,NaN
1,4287,NaN,NaN,Полная занятость,InformationTechnology,2024-07-26 06:02:38+00:00,Высшее,Информационные технологии,NaN,знание технологий обмена и опыт работы с механ...,...,"{""companycode"":""1227700184161"",""email"":""info@v...",[],[],[],"[http, web]","[http, web]",NaN,NaN,"http,web","http,web"
2,4459,NaN,NaN,Полная занятость,InformationTechnology,2024-07-26 10:57:10+00:00,Среднее профессиональное,NaN,NaN,,...,"{""companycode"":""1165958087228"",""email"":""info@p...",[],[],[],NaN,NaN,NaN,NaN,NaN,NaN
3,5311,NaN,NaN,Полная занятость,Transport,2024-07-25 23:02:41+00:00,Высшее,NaN,NaN,,...,"{""companycode"":""9fdc1f30-7578-11ea-bdff-736ab1...",[],[],[],[BTL],NaN,[BTL],NaN,BTL,BTL
4,7585,NaN,NaN,Полная занятость,Industry,2024-07-25 05:47:16+00:00,Высшее,Маркетинг,NaN,креативное мышление (интересные идеи и решения...,...,"{""companycode"":""1202200000650"",""email"":""697014...",[],"[{""hard_skill_name"":""умение работать в условия...","[{""owner_id"":""dd3835c5-40b9-11ee-ad0a-9586bb63...",NaN,NaN,NaN,NaN,NaN,NaN


Определяю квалификацию

In [112]:
df_qual_1 = ds_vac_all["position_requirements"].str.extract(
    r'(?P<qual_1>Стажер|мидл|сеньор|джун|intern|jr|Junior|Middle|Senior|Руководитель|Lead)',
    flags=re.I, expand=True).apply(lambda ql: ql.str.lower().str.strip())
df_qual_1

,qual_1
0,NaN
1,NaN
2,NaN
3,NaN
4,NaN
...,...
951,NaN
952,NaN
953,NaN
954,NaN


In [113]:
df_qual_2 = ds_vac_all["vacancy_name"].str.extract(
    r'(?P<qual_2>Стажер|мидл|сеньор|джун|intern|jr|Junior|Middle|Senior|Руководитель|Lead)',
    flags=re.I, expand=True).apply(lambda ql: ql.str.lower().str.strip())
df_qual_2

,qual_2
0,middle
1,NaN
2,NaN
3,NaN
4,NaN
...,...
951,NaN
952,NaN
953,NaN
954,NaN


In [114]:
temp_df_qual = pd.merge(df_qual_2,df_qual_1, left_index=True, right_index=True)
temp_df_qual['qual'] = temp_df_qual.apply(
    lambda row: row["qual_1"] if pd.isnull(row['qual_2']) else row['qual_2'],
    axis=1
)
temp_df_qual['qual'].isnull().sum()

936

In [115]:
ds_vac_all["qualification"] = temp_df_qual['qual']

In [116]:
ds_vac_all["qualification"]=ds_vac_all["qualification"].map({"стажер":"junior","джун":"junior","jr":"junior","intern":"junior","junior":"junior",
                                          "мидл":"middle","middle":"middle",
                                          "сеньор":"senior","руководитель":"senior","lead":"senior","senior":"senior"})

In [137]:
ds_vac_all["qualification"].fillna("Not_qualification", inplace=True)

In [117]:
ds_vac_all["salary_mean"] = (ds_vac_all["salary_min"]+ds_vac_all["salary_max"])/2

Очищаю название вакансий

In [143]:
def func_clr_job(job):
    if isinstance(job, str):
        job_list_ = re.split('[\\\/|]+',job.strip(' .?,!:;!@#$%^&*()-_+><[]{}\|/\D"'),flags=re.I)
        job_ = ' '.join(job_list_[-1].replace("-"," ").split())
        if len(job_) !=0:
            string = []
            for part in job_.split(" "):
                string.append("".join([c for c in part if c.isalpha()]))
            return " ".join(string)

ds_vac_all["vacancy_name"] =ds_vac_all.vacancy_name.str.lower().str.replace("(\()[^()]+(\))", '', regex=True)
ds_vac_all["vacancy_name"] = ds_vac_all["vacancy_name"].apply(lambda x: func_clr_job(x))

In [118]:
ds_vac_all["busy_type_"]=ds_vac_all["busy_type"].map({"Полная занятость":"full-time","Удаленная":"remote","Временная":"temporary","Частичная занятость":"part-time","Стажировка":"internship"})
ds_vac_all["busy_type"].isnull().sum()

0

In [119]:
ds_vac_all["busy_type"] = ds_vac_all["busy_type_"]

Собираю итоговый датасет

In [144]:
df_vac2 = ds_vac_all[["vacancy_name","busy_type","qualification","regionName","federalDistrictCode","skills_all","salary_min","salary_max"]]
df_vac2.head()

,vacancy_name,busy_type,qualification,regionName,federalDistrictCode,skills_all,salary_min,salary_max
0,разработчик с,full-time,middle,Курганская область,5.0,NaN,150000.0,0.0
1,разработчик с,full-time,Not_qualification,Свердловская область,5.0,"http,web",180000.0,0.0
2,системный аналитик,full-time,Not_qualification,Пермский край,4.0,NaN,60000.0,0.0
3,маркетолог,full-time,Not_qualification,Приморский край,7.0,BTL,60000.0,60000.0
4,маркетолог,full-time,Not_qualification,Алтайский край,6.0,NaN,70000.0,120000.0


In [145]:
df_vac2.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 956 entries, 0 to 955
Data columns (total 8 columns):
 #   Column               Non-Null Count  Dtype  
---  ------               --------------  -----  
 0   vacancy_name         956 non-null    object 
 1   busy_type            956 non-null    object 
 2   qualification        956 non-null    object 
 3   regionName           941 non-null    object 
 4   federalDistrictCode  956 non-null    float64
 5   skills_all           247 non-null    object 
 6   salary_min           956 non-null    float64
 7   salary_max           947 non-null    float64
dtypes: float64(3), object(5)
memory usage: 59.9+ KB


In [146]:
df_vac2.to_csv(r"./dataset_trudvsem.csv", sep ="|",
                header=["name_job","type_busy", "qualification", "location","federalDistrictCode","skills","salary_min","salary_max"],
                index=False)